# SQL Practice
This notebook designed to practice basic SQL queries on the famild.db.

In [1]:
%load_ext sql

In [2]:
%sql sqlite:///family.db

In [26]:
%%sql

select first_name as MILF
from person, child_of
where mother_id = id
and original_surname = new_surname
and (birth_date > 50);

 * sqlite:///family.db
Done.


MILF
Old Feminist Mom
Levs grandmother
Levs mom


In [27]:
%%sql

-- without subquery - use join

SELECT first_name
FROM person
JOIN child_of ON person.id = child_of.mother_id
WHERE person.original_surname = person.new_surname
AND (strftime('%Y', 'now') - strftime('%Y', birth_date)) > 50;


 * sqlite:///family.db
Done.


first_name
Old Feminist Mom
Levs grandmother
Levs mom


In [28]:
%%sql

-- again without join

select first_name
from person
where original_surname = new_surname
and (strftime('%Y', 'now') - strftime('%Y', birth_date)) > 50
and id in
    (select mother_id
    from child_of);

 * sqlite:///family.db
Done.


first_name
Old Feminist Mom
Levs grandmother
Levs mom


In [29]:
%%sql
-- sqllite dont support!!
select count(child_id)
from child_of
group by (father_id, mother_id)

 * sqlite:///family.db
(sqlite3.OperationalError) row value misused
[SQL: -- sqllite dont support!!
select count(child_id)
from child_of
group by (father_id, mother_id)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [30]:
%%sql

SELECT father_id, mother_id, COUNT(child_id) as num_children
FROM child_of
GROUP BY father_id, mother_id;

 * sqlite:///family.db
Done.


father_id,mother_id,num_children
1,2,2
1,7,1
3,4,2
11,111,1
12,122,1


In [31]:
%%sql

-- now for maried couples only:

select m_id, w_id, count(child_id) as num_children
from  marriage, child_of
group by mother_id, father_id
having m_id = father_id and w_id = mother_id


 * sqlite:///family.db
Done.


m_id,w_id,num_children
1,2,10


When you are trying to perform a query on multiple tables using an implicit JOIN (by listing the tables separated by commas), you'll need to specify the conditions for how the tables relate to each other in the WHERE clause. This is known as an implicit join condition.

Here's the corrected query:

```sql

%%sql

SELECT m.m_id, m.w_id, COUNT(c.child_id) as num_children
FROM marriage m, child_of c
WHERE m.m_id = c.father_id AND m.w_id = c.mother_id
GROUP BY m.m_id, m.w_id;
```

Changes made:

    Added table aliases for clarity: m for marriage and c for child_of.
    Moved the conditions specifying the relationship between the marriage and child_of tables to the WHERE clause.

The resulting query should give you the count of children for each married couple.

In [32]:
%%sql

SELECT m.m_id, m.w_id, COUNT(c.child_id) as num_children
FROM marriage m, child_of c
WHERE m.m_id = c.father_id AND m.w_id = c.mother_id
GROUP BY m.m_id, m.w_id;

 * sqlite:///family.db
Done.


m_id,w_id,num_children
1,2,2
3,4,2
11,111,1
12,122,1


1. 11 - grandgrandfather
2. 12 - Lev's father
3. 3  - Lev
4. 5  - Lev's son

In [33]:
%%sql
-- lets creat 3 generation's family name
insert into person values 
    ('11', 'Levs grndfather','Ayzen', 'Ayzen', 1900-01-01),
    ('111', 'Levs grandmother', 'Ayzen', 'Ayzen', 1901-10-10),
    ('12', 'Yossi', 'Ayzen', 'Ayzen', 1929-10-10),
    ('122', 'Levs mom','Ayzen', 'Ayzen', 1929-10-10)
    ;

insert into child_of values
    ('12', '11', '111' ), -- yosi
    ('3', '12','122'); -- lev

insert into marriage values
    ('11', '111', '1920-01-01'), --grabdoarabts
    ('12', '122', '1960-01-01'); -- parants

 * sqlite:///family.db
(sqlite3.IntegrityError) UNIQUE constraint failed: person.id
[SQL: -- lets creat 3 generation's family name
insert into person values 
    ('11', 'Levs grndfather','Ayzen', 'Ayzen', 1900-01-01),
    ('111', 'Levs grandmother', 'Ayzen', 'Ayzen', 1901-10-10),
    ('12', 'Yossi', 'Ayzen', 'Ayzen', 1929-10-10),
    ('122', 'Levs mom','Ayzen', 'Ayzen', 1929-10-10)
    ;]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [34]:
%%sql

select original_surname as old_surname
from person, child_of
where id = child_id
group by original_surname
having original_surname not exists in
    select original_surname
    from person, child_of
    where id = child_id
    group by original_surname
    not in
        select original_surname
        from person, child_of
        where id = child_id;

 * sqlite:///family.db
(sqlite3.OperationalError) near "exists": syntax error
[SQL: select original_surname as old_surname
from person, child_of
where id = child_id
group by original_surname
having original_surname not exists in
    select original_surname
    from person, child_of
    where id = child_id
    group by original_surname
    not in
        select original_surname
        from person, child_of
        where id = child_id;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [35]:
%%sql

SELECT p1.original_surname AS old_surname
FROM person p1
WHERE EXISTS ( 
    SELECT 1 FROM child_of c1 WHERE c1.child_id = p1.id 
    AND EXISTS ( 
        SELECT 1 FROM child_of c2 WHERE (c2.child_id = c1.father_id OR c2.child_id = c1.mother_id)
        AND EXISTS (
            SELECT 1 FROM child_of c3 WHERE (c3.child_id = c2.father_id OR c3.child_id = c2.mother_id)
        )
    )
)
GROUP BY p1.original_surname;

 * sqlite:///family.db
Done.


old_surname
Ayzen
Doe
Same
Smith


In [11]:
%%sql

select distinct son.original_surname
from person as grandfather, person as father, person as son,
    child_of as son, child_of as father
where grandfather.original_surname = father.original_surname
  and father.original_surname = son.original_surname
  and father.id = son.father_id
  and grandfather.id = father.father_id;

 * sqlite:///family.db
Done.


original_surname
Smith
Ayzen


In [37]:
%%sql

select brother1.child_id, brother2.child_id
from child_of as brother1, child_of as brother2
where brother1.child_id <> brother2.child_id
 and brother1.father_id = brother2.father_id
 and brother1.mother_id <> brother2.mother_id
union
select brother1.child_id, brother2.child_id
from child_of as brother1, child_of as brother2
where brother1.child_id <> brother2.child_id
 and brother1.father_id <> brother2.father_id
 and brother1.mother_id = brother2.mother_id;

 * sqlite:///family.db
Done.


child_id,child_id_1
1,8
2,8
8,1
8,2


wrong - union add dublicate!!!

In [24]:
%%sql

select brother1.child_id, brother2.child_id
from child_of as brother1, child_of as brother2
where (brother1.child_id <> brother2.child_id)
 and ((brother1.father_id = brother2.father_id
         and brother1.mother_id <> brother2.mother_id)
    or
    (brother1.father_id <> brother2.father_id
     and brother1.mother_id = brother2.mother_id));

 * sqlite:///family.db
Done.


child_id,child_id_1
1,8
2,8
8,1
8,2


still wrong! must aliminate dups in couples! We'll use brother1.id 

In [38]:
%%sql

select brother1.child_id, brother2.child_id
from child_of as brother1, child_of as brother2
where (brother1.child_id < brother2.child_id)
 and ((brother1.father_id = brother2.father_id
         and brother1.mother_id <> brother2.mother_id)
    or
    (brother1.father_id <> brother2.father_id
     and brother1.mother_id = brother2.mother_id));

 * sqlite:///family.db
Done.


child_id,child_id_1
1,8
2,8
